# Idea for reversal of impact chain - glaciers



**Newer version: v2024-03-26**

**Some additional information:**
- we used LOWESS fits
    - originally done to create these figures here https://github.com/lilianschuster/glacier-model-projections-until2300
        - LOWESS fits done with the MOEPY package -> https://ayrtonb.github.io/Merit-Order-Effect/ug-08-lowess-quick-start/#quantile
        - we computed percentiles from 1 to 99th percentile in "1%" steps, but if you want others, we can create other percentiles... 
- global mean warming here defined as done in IPCC AR6 WG1, that means:
    - assume 0.69Â°C warming from preindustrial levels (1850-1900) to 1986-2005 for every GCM, then use the warming as given by individual GCMs ... 
- That means we use some glacier model data from in total two glacier models (will be updated to have three glacier models...). 
- Fits were done for all CMIP5 and CMIP6 GCMs going until 2100
    - some glacier models have done projections with more GCMs than others .... 
    - should be sufficient to analyse if this could be a good example


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

plt.rc('font', size=18)

pd_raw_data_sel = pd.read_csv('glacier_model_proj_from_GCMs_until_2100_test_RGI13-14-15_2100_v2024-03-26_OGGM_PyGEM.csv', index_col=[0])
pd_fit = pd.read_csv('lowess_fit_from_GCMs_until_2100_RGI13-14-15_2100_OGGM_PyGEM_v2024-03-26.csv', index_col=[0])

quantiles_str = []
quantiles = np.arange(0.01,1,0.01)
for i in quantiles:
    _str = f'q{i:0.2f}' #.format(i)  # Format the percentile string with leading zeros
    quantiles_str.append(_str)
    
# revert the columns 
pd_fit.index = pd_fit.deltaTemp.values.round(2)
pd_fit_v = pd_fit[quantiles_str].T
pd_fit_v['q'] = quantiles


assert len(pd_fit.frac.unique())==1
frac = pd_fit.frac.unique()[0].round(2)
colors = sns.color_palette('viridis', n_colors=100)
colors_2 = sns.color_palette('rocket_r', n_colors=12)
plt.figure(figsize=(20,10))
plt.subplot(121)



for c,q in zip(colors,quantiles_str): 
    if q == 'q0.50':
        sns.lineplot(data=pd_fit, x='deltaTemp', y=q, color='black', label=q, lw=3, zorder=6)
    else:
        sns.lineplot(data=pd_fit, x='deltaTemp', y=q, color=c, label=q, alpha = 0.8, zorder=5)
ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()
ax.get_legend().remove()
leg = ax.legend([handles[0],handles[49], handles[-1]], [labels[0],labels[49], labels[-1]], title=f'LOWESS (frac={frac})\nquantile regression')
plt.ylabel('Remaining glacier ice mass in HMA\n(% rel. to 2020)')
plt.xlabel('Global mean temperature change (Â°C)')

_pd_sel = pd_raw_data_sel.loc[pd_raw_data_sel.region == 'RGI13-14-15']
_pd_sel = _pd_sel.loc[_pd_sel.year == 2100]
sns.scatterplot(data= _pd_sel, x= 'global_temp_ch_2071-2100_preindustrial', y='rel_ice_%_2020', hue='model', zorder=10)
handles, labels = ax.get_legend_handles_labels()
ax.get_legend().remove()
leg2 = ax.legend(handles[-2:], labels[-2:], loc = 'lower left', title = 'glacier model\nprojections')
ax.add_artist(leg)
for j,t in enumerate(np.arange(1,6.5,0.5)):
    plt.axvline(t, lw=0.5,  color=colors_2[j])

plt.subplot(122)
for j,t in enumerate(np.arange(1,6.5,0.5)):
    sns.scatterplot(data=pd_fit_v, x= 'q', y= t, color=colors_2[j], label=f'{t}Â°C')
plt.legend(loc='upper left', bbox_to_anchor=(1,1), title= 'GMT')
plt.xlabel('quantile')
plt.ylabel('Remaining glacier ice mass in HMA\n(% rel. to 2020)');

In [ ]:
pd_fit_v

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde


# for 1.5
pd_fit_v_sel = pd_fit_v[1.5]

# Assuming you have a list of quantile values called quantile_values
quantile_values = np.array(pd_fit_v_sel.values)

# Estimate the density function using kernel density estimation
kde = gaussian_kde(quantile_values)

# Generate points to plot the estimated density function
x = np.linspace(np.min(quantile_values), np.max(quantile_values), 1000)
density_estimate = kde(x)

# Plot the estimated density function
plt.plot(x, density_estimate)
plt.xlabel('Value')
plt.ylabel('Density')
plt.title('Estimated Probability Density Function')
plt.show()

In [ ]:
pd_fit_v

In [ ]:
sns.kdeplot(data=pd_fit_v[np.arange(1.5,5,0.5)], palette=sns.color_palette('rocket_r', n_colors=7), legend=True, cut=0,
           fill=True, alpha=0.5,lw=2)
plt.xlabel('Remaining glacier ice mass in HMA\n(% rel. to 2020)')
ax = plt.gca()
leg = ax.get_legend()
#handles, labels = ax.get_legend_handles_labels()
leg.set_bbox_to_anchor((1.4,1))
leg.set_title('GMT (Â°C)')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# Assuming corrected q and d arrays of the same length
q = np.linspace(0.01, 0.991, 99)  # Quantiles from 0.01 to 0.99
d = pd_fit_v_sel.values # np.linspace(1, 99, 99)  # Corresponding data values, adjust as per your data

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(q, d, marker='o')
plt.title('Frequency Distribution from Quantiles')
plt.xlabel('Quantiles')
plt.ylabel('Data Values')
plt.grid(True)
plt.show()


### added by peter

In [1]:
import glob
import xarray as xr
import collections
import seaborn as sns
import matplotlib
plt.rc('font', size=12)
matplotlib.rcParams['figure.figsize'] = (4,3)

NameError: name 'plt' is not defined

In [ ]:
pd_fit = pd.read_csv('lowess_fit_from_GCMs_until_2100_RGI13-14-15_2100_OGGM_PyGEM_GloGEM_v2024-03-26.csv', index_col=[0])

In [ ]:
# store pd_fit in an xarray
quantiles = xr.DataArray(pd_fit_v.T.values[:-1], dims=['gmt','q'], coords=dict(gmt=pd_fit_v.columns.values[:-1], q=[float(s[1:]) for s in pd_fit_v.index.values]))

In [ ]:
def prob_greater_at_gmt(g, thresh):
    '''
    returns the probability of keeping the threshold amount of glacier at given GMT g
    '''
    y = quantiles.sel(gmt=g, method='nearest')
    return 1 - quantiles.q.values[np.abs(y - thresh).argmin()]

In [ ]:
gmt_axis = np.arange(1,4,0.1)
p = np.array([prob_greater_at_gmt(g, 50) for g in gmt_axis])
plt.plot(gmt_axis, p)
plt.xlabel('GMT')
plt.ylabel('probability of keeping\n50% of glaciers')
plt.tight_layout()

GMT:
1986-2005 vs 1850-1900 
1986-2005 should be 0.69K warmer than preindustrial
gmt -= gmt.loc[1986-2005].mean() + 0.69

In [ ]:
def prob_greater_in_scenario(scen, thresh):
    '''
    returns the probability of crossing thresh in given scenario
    '''
    # get all gmt values in 2100 from the 2237 runs
    # bin GMT in 0.1 K steps
    # and count occurences in bins
    counter = collections.Counter(gmt.loc[scen].round(1).values)
    p = np.array([])
    # for each bin add the probability of crossing thresh at GMT level "count"-times
    for g,count in counter.items():
        p = np.append(p, [prob_greater_at_gmt(g, thresh)]*count)
    return p.mean()

In [ ]:
year = 2100

In [ ]:
gmt = xr.open_dataset(f'gmt_in_{year}.nc')['gmt']

In [ ]:
# calculate the probability of keeping X% of the glacier for all 1500 scenarios
p_greater = xr.DataArray(dims=['scenario', 'thresh'], coords=dict(scenario=gmt.scenario.values, thresh=np.arange(40,70,10,'int')))
for thresh in p_greater.thresh.values:
    for scen in p_greater.scenario.values:
        p_greater.loc[scen, thresh] = prob_greater_in_scenario(scen, thresh)
xr.Dataset({'p_greater':p_greater}).to_netcdf(f'p_greater_in_{year}.nc')

In [ ]:
p_greater = xr.open_dataset(f'p_greater_in_{year}.nc')['p_greater']

In [ ]:
for thresh in p_greater.thresh.values[:]:
    plt.scatter(gmt.mean('run'),p_greater.loc[:,thresh], label=f'{thresh}')
plt.xlabel('GMT averaged over runs of the scenario')
plt.ylabel('probability of only\n... of the glacier remaining')
plt.legend(title='remaining')

In [ ]:
# identify scenarios for which there is a 90% chance of keeping 40% of the glaciers
boundary_scenarios = p_greater.scenario[np.abs(p_greater.loc[:,40] - 0.90) < 0.01]

In [ ]:
# identify scenarios for which there is a 90% chance of keeping 50% of the glaciers
boundary_scenarios = p_greater.scenario[np.abs(p_greater.loc[:,50] - 0.80) < 0.01]

In [ ]:
# GMT distributions for boundary scenarios in 2100
fig,ax = plt.subplots(nrows=1, figsize=(4,3))
for scen in boundary_scenarios:
    sns.kdeplot(gmt.loc[scen], color='gray', linewidth=0.3)
    ax.axvline(gmt.loc[scen].median(), color='gray', linewidth=0.3)
ax.set_xlabel('GMST')

In [ ]:
# load fair emissions
emissions = pd.read_table('../Tier2_scenarios_emissions.csv', sep=',')
co2 = emissions.loc[(emissions.Variable == 'Emissions|CO2')]
co2 = xr.DataArray(co2.iloc[:,5:].values, dims=['scen','year'], coords=dict(scen=co2.Scenario, year=np.array(co2.columns[5:], 'int')))
ch4 = emissions.loc[(emissions.Variable == 'Emissions|CH4')]
ch4 = xr.DataArray(ch4.iloc[:,5:].values, dims=['scen','year'], coords=dict(scen=ch4.Scenario, year=np.array(ch4.columns[5:], 'int')))
cum_co2 = (co2.loc[:,2020:2100].rolling(year=2).sum() * 0.5 * 10).sum('year') / 1000

In [ ]:
# select scenarios that stabilize in 2100
# -> no overshoot efore 2100
# -> potential for keeping the glacier mass later on as well
selected_boundary_scenarios = boundary_scenarios[(co2.loc[boundary_scenarios].loc[:,2030] <= co2.loc[boundary_scenarios].loc[:,2020])]

In [ ]:
gmt_median = xr.open_dataset('gmt_median.nc')['gmt']

In [ ]:

fig,axes = plt.subplots(nrows=1, ncols=2, figsize=(8,3))
axes[1].axhline(0, color='k')
for scenarios, color in zip([boundary_scenarios, selected_boundary_scenarios], ['gray','orange']):
    for scen in scenarios.values:
        y = gmt_median.loc[scen,:2100]
        axes[0].plot(y.year, y, label=scen, color=color)
        y = co2.loc[scen,:2100] / 1000
        axes[1].plot(y.year, y, color=color)
    
axes[0].set_ylabel('GMST [K]')
axes[1].set_ylabel('CO2 emissions [Gt/yr]')
plt.tight_layout() 

In [ ]:
# get cumulative CO2 emissions until 2100 in the orange scenario
CO2_budget = cum_co2.loc[selected_boundary_scenarios].values.mean()
CO2_budget

In [ ]:
cum_co2.loc[selected_boundary_scenarios].values.std()